In [1]:
import os
os.chdir(os.path.join('..', '..', '..'))
print(os.getcwd())

/Users/eharkin/Documents/projects/5HTGIF


In [ ]:
from __future__ import division

import pickle

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import seaborn as sns
from scipy import stats

from grr.cell_class import (
    Cell,
    Recording,
    subtract_baseline,
    subtract_leak,
)
from grr import pltools
from grr.CurveFit import fit_decay_curve, fit_gating_curve, plot_linear_fit
from grr.Tools import dashedBorder, timeToIndex

In [ ]:
IMG_PATH = os.path.join('figs', 'ims', 'writeup', 'fig1')
NOTEBOOK_PATH = os.path.join('figs', 'scripts', 'writeup')

plt.style.use(os.path.join(NOTEBOOK_PATH, 'publication_figure_style.dms'))

In [ ]:
# Load V-steps files for sample pharma traces.
baseline = Cell().read_ABF('./data/raw/figdata/18411010.abf')[0]
baseline.set_dt(0.1)
TEA = Cell().read_ABF('./data/raw/figdata/18411013.abf')[0]
TEA.set_dt(0.1)
TEA_4AP = Cell().read_ABF('./data/raw/figdata/18411015.abf')[0]
TEA_4AP.set_dt(0.1)

GATING_PATH = './data/raw/5HT/gating/'

# Load gating data
gating = Cell().read_ABF([GATING_PATH + '18411002.abf',
                          GATING_PATH + '18411010.abf',
                          GATING_PATH + '18411017.abf',
                          GATING_PATH + '18411019.abf',
                          GATING_PATH + 'c0_inact_18201021.abf',
                          GATING_PATH + 'c1_inact_18201029.abf',
                          GATING_PATH + 'c2_inact_18201034.abf',
                          GATING_PATH + 'c3_inact_18201039.abf',
                          GATING_PATH + 'c4_inact_18213011.abf',
                          GATING_PATH + 'c5_inact_18213017.abf',
                          GATING_PATH + 'c6_inact_18213020.abf',
                          GATING_PATH + '18619018.abf',
                          GATING_PATH + '18614032.abf'])

In [ ]:
gating[0].plot()

In [ ]:
print(gating[0][1, 0, :])
print(gating[0][1, 20000, :])
print(gating[0][1, 40000, :])

# Subtract baseline/leak

In [ ]:
baseline = subtract_baseline(baseline, slice(1000, 2000), 0)
baseline = subtract_leak(baseline, slice(1000, 2000), slice(3000, 3400))
baseline.set_dt(0.1)

TEA_4AP = subtract_baseline(TEA_4AP, slice(1000, 2000), 0)
TEA_4AP = subtract_leak(TEA_4AP, slice(1000, 2000), slice(3000, 3400))
TEA_4AP.set_dt(0.1)

gating_tmp = []
for rec in gating:
    baseline_subtracted = subtract_baseline(rec, slice(1000, 2000), 0)
    leak_subtracted = subtract_leak(baseline_subtracted, slice(1000, 2000), slice(3000, 3400))
    leak_subtracted.set_dt(0.1)
    gating_tmp.append(leak_subtracted)
gating = gating_tmp
del gating_tmp, baseline_subtracted, leak_subtracted 

# Fit decay

In [ ]:
step_amplitude = 50  # 70mV (-70 to -20)

IA_inactivation = {
    'traces': [],
    'fitted_data': [],
    'fitted_curves': [], 
    'range_fitted': [],
    'inactivation_taus': [],
    'peak_latencies': [],
    'peak_amplitudes': [],
    'steady_state_amplitudes': []
}

peak_fit_params = {
    'stimulus_start_time': 2606.2,
    'steady_state_time': 3606.2,
    'peak_slice': slice(26085, 30000),
    'decay_slice': slice(26250, 27500)
}

for i in range(len(gating)):
    
    # Find peak latency
    IA_inactivation['peak_latencies'].append(
        np.argmax(gating[i][0, peak_fit_params['peak_slice'], -1]) * 0.1 
        + peak_fit_params['peak_slice'].start * 0.1 
        - peak_fit_params['stimulus_start_time']
    )
    
    # Get peak amplitude.
    IA_inactivation['peak_amplitudes'].append(
        gating[i][
            0, 
            np.argmax(gating[i][0, peak_fit_params['peak_slice'], -1]) + peak_fit_params['peak_slice'].start, 
            -1
        ]
    )
    
    # Get steady-state amplitude
    IA_inactivation['steady_state_amplitudes'].append(
        gating[i][0, timeToIndex(peak_fit_params['steady_state_time'], 0.1)[0], -1]
    )
    
    # Fit decay
    IA_inactivation['range_fitted'].append([peak_fit_params['decay_slice'].start * 0.1, peak_fit_params['decay_slice'].stop * 0.1])

    t_vec = np.arange(0, gating[i].shape[1], 0.1)[:gating[i].shape[1]]
    fitted_tmp = gating[i][0, peak_fit_params['decay_slice'], -1]
    IA_inactivation['fitted_data'].append(np.array(
        [fitted_tmp,
        t_vec[peak_fit_params['decay_slice']]]
    ))

    IA_inactivation['traces'].append(np.array([gating[i][0, :, -1], t_vec]))

    p_tmp, fitted_tmp = fit_decay_curve(
        fitted_tmp,
        [fitted_tmp[0] - fitted_tmp[-1], fitted_tmp[-1], 20],
        dt = 0.1
    )

    IA_inactivation['inactivation_taus'].append(p_tmp[2])
    IA_inactivation['fitted_curves'].append(np.array(
        [fitted_tmp[0, :],
        np.linspace(peak_fit_params['decay_slice'].start * 0.1, peak_fit_params['decay_slice'].stop * 0.1, len(fitted_tmp[0, :]))]
    ))

    # Diagnostic plot of decay fit.
    plt.figure()
    plt.plot(t_vec, gating[i][0, :, -1], 'k-', lw=0.5)
    #plt.plot(t_vec[peak_fit_params['decay_slice']], gating[i][0, peak_fit_params['decay_slice'], -1], 'r-')
    plt.plot(
        np.linspace(
            peak_fit_params['decay_slice'].start * 0.1, peak_fit_params['decay_slice'].stop * 0.1, len(fitted_tmp[0, :])
        ),
        fitted_tmp[0, :], '--', color = 'gray', lw=4
    )
    plt.xlim(2600, 3000)
    plt.show()


for key in IA_inactivation:
    IA_inactivation[key] = np.array(IA_inactivation[key])
print('IA inactivation tau {:.1f} +/- {:.1f} (mean +/- SD)'.format(
    IA_inactivation['inactivation_taus'].mean(), IA_inactivation['inactivation_taus'].std()
))

#with open(PROCESSED_PATH + 'inactivation_fits.dat', 'wb') as f:
#    pickle.dump(IA_inactivation, f)

In [ ]:
IA_inactivation.keys()

In [ ]:
for dataset in ['peak_amplitudes', 'peak_latencies', 'steady_state_amplitudes', 'inactivation_taus']:
    print('{:>25} {:>10.3} +/- {:>5.3}'.format(
        dataset, 
        np.mean(IA_inactivation[dataset]), 
        stats.sem(IA_inactivation[dataset])
    ))

## Summary statistics for quantities in nS

In [ ]:
print('Peak amplitudes (nS) {:>20.3} +/- {:>5.3}'.format(
    np.mean(IA_inactivation['peak_amplitudes'] / step_amplitude),
    stats.sem(IA_inactivation['peak_amplitudes'] / step_amplitude)
))
print('Steady state amplitudes (nS) {:>20.3} +/- {:>5.3}'.format(
    np.mean(IA_inactivation['steady_state_amplitudes'] / step_amplitude),
    stats.sem(IA_inactivation['steady_state_amplitudes'] / step_amplitude)
))

In [ ]:
fit_example_no = 5
bg_tr_alpha = 0.2

spec_outer = gs.GridSpec(1, 3, width_ratios=[1, 0.7, 0.7], wspace=0.5, bottom=0.2, right=0.95)
peak_params_spec = gs.GridSpecFromSubplotSpec(1, 2, spec_outer[:, 1], wspace=1.2)
off_peak_params_spec = gs.GridSpecFromSubplotSpec(1, 2, spec_outer[:, 2], wspace=1.2)

plt.figure(figsize=(4, 1.5))

wc_ax = plt.subplot(spec_outer[:, 0])
plt.plot(
    (IA_inactivation['traces'][:, 1, :].T - 2606.2) * 1e-3, 
    IA_inactivation['traces'][:, 0, :].T * 1e-3,
    'k-', lw=0.5, alpha=bg_tr_alpha
)
plt.plot(
    (IA_inactivation['traces'][fit_example_no, 1, :].T - 2606.2) * 1e-3, 
    IA_inactivation['traces'][fit_example_no, 0, :].T * 1e-3,
    'k-', lw=0.5, zorder=100
)
plt.plot(
    (peak_fit_params['steady_state_time'] - peak_fit_params['stimulus_start_time']) * 1e-3,
    IA_inactivation['steady_state_amplitudes'][fit_example_no] * 1e-3,
    'ko', zorder=101
)
plt.xlim(-0.05, 1.05)
plt.ylim(-0.1, 1.6)
plt.xlabel('Time from step (s)')
plt.ylabel('Leak subtracted current (nA)')
sns.despine(ax=plt.gca(), trim=True)

wc_ins = inset_axes(wc_ax, '60%', '50%', loc='upper right', borderpad=1.2)
plt.plot(
    (IA_inactivation['traces'][:, 1, :].T - 2606.2) * 1e-3, 
    IA_inactivation['traces'][:, 0, :].T * 1e-3,
    'k-', lw=0.5, alpha=bg_tr_alpha
)
plt.plot(
    (IA_inactivation['traces'][fit_example_no, 1, :].T - 2606.2) * 1e-3, 
    IA_inactivation['traces'][fit_example_no, 0, :].T * 1e-3,
    'k-', lw=0.5, zorder=100
)
plt.plot(
    IA_inactivation['peak_latencies'][fit_example_no] * 1e-3,
    IA_inactivation['peak_amplitudes'][fit_example_no] * 1e-3,
    'ko', zorder=101
)
plt.plot(
    (
        np.linspace(
            peak_fit_params['decay_slice'].start * 0.1, 
            peak_fit_params['decay_slice'].stop * 0.1, 
            len(fitted_tmp[0, :])
        ) 
        - peak_fit_params['stimulus_start_time']
    ) * 1e-3,
    IA_inactivation['fitted_curves'][fit_example_no, 0, :] * 1e-3, 
    '--', color = 'green', alpha=0.7, lw=3, zorder=101
)
plt.xticks([])
plt.yticks([])
plt.xlim(-0.02, 0.15)
plt.ylim(-0.05, 1.5)
dashedBorder(wc_ins, lw=1.5)
mark_inset(wc_ax, wc_ins, 1, 4, ls='--', color='gray', lw=1.5)

plt.subplot(peak_params_spec[:, 0])
plt.ylim(0, max(IA_inactivation['peak_amplitudes'] / step_amplitude) * 1.05)
sns.swarmplot(y=IA_inactivation['peak_amplitudes'] / step_amplitude, size=5, clip_on=False)
plt.ylabel('Peak conductance (nS)')
sns.despine(ax=plt.gca(), trim=True)
plt.xticks([])

plt.subplot(peak_params_spec[:, 1])
plt.ylim(0, max(IA_inactivation['peak_latencies']) * 1.05)
sns.swarmplot(y=IA_inactivation['peak_latencies'], size=5, clip_on=False)
plt.ylabel('Time to peak (ms)')
sns.despine(ax=plt.gca(), trim=True)
plt.xticks([])

plt.subplot(off_peak_params_spec[:, 0])
plt.ylim(0, max(IA_inactivation['steady_state_amplitudes'] / step_amplitude) * 1.05)
sns.swarmplot(y=IA_inactivation['steady_state_amplitudes'] / step_amplitude, size=5, clip_on=False)
plt.ylabel('Steady-state conductance (nS)')
sns.despine(ax=plt.gca(), trim=True)
plt.xticks([])

plt.subplot(off_peak_params_spec[:, 1])
plt.ylim(0, max(IA_inactivation['inactivation_taus']) * 1.05)
sns.swarmplot(y=IA_inactivation['inactivation_taus'], size=5, clip_on=False)
plt.ylabel('Decay time constant (ms)')
sns.despine(ax=plt.gca(), trim=True)
plt.xticks([])

plt.tight_layout()

if IMG_PATH is not None:
    plt.savefig(os.path.join(IMG_PATH, 'ser_kinetics.png'))
    
plt.show()

In [ ]:
Kslow_xlim = (2450, 8500)

spec_pharm = gs.GridSpec(2, 1, height_ratios=[1, 0.2], hspace=0)

plt.figure(figsize=(2, 2))

pharm_ax = plt.subplot(spec_pharm[0, :])
plt.plot(
    baseline.t_mat[0, :, -3],
    baseline[0, :, -3],
    linewidth = 0.5, color = 'k',
    label = 'Baseline'
)
plt.plot(
    TEA_4AP.t_mat[0, :, -3],
    TEA_4AP[0, :, -3],
    linewidth = 0.5, color = (0.9, 0.2, 0.2),
    label = 'TEA + 4AP'
)
plt.text(3800, 200, '+TTX', ha = 'center', va = 'center', size = 'small')
plt.axhline(0, ls = '--', lw = 0.5, color = 'k', dashes = (10, 10))
plt.xlim(Kslow_xlim[0], 4500)
plt.ylim(-100, 550)
pltools.add_scalebar(
    'ms', 'pA', anchor = (1, 0.35), y_size = 100, x_size = 500,
    bar_space = 0, x_label_space = -0.02
)
plt.legend()

pharm_ins = inset_axes(pharm_ax, '30%', '50%', loc = 'upper left', borderpad = 2.7)
plt.plot(
    baseline.t_mat[0, :, -3],
    baseline[0, :, -3],
    linewidth = 0.5, color = 'k',
    label = 'Baseline'
)
plt.plot(
    TEA_4AP.t_mat[0, :, -3],
    TEA_4AP[0, :, -3],
    linewidth = 0.5, color = (0.9, 0.2, 0.2),
    label = 'TEA + 4AP'
)
plt.xlim(Kslow_xlim[0] + 100, Kslow_xlim[0] + 300)
plt.ylim(-70, 500)
dashedBorder(pharm_ins)
pltools.hide_ticks()
mark_inset(pharm_ax, pharm_ins, loc1 = 1, loc2 = 4, ls = '--', color = 'gray', lw = 0.5)

plt.subplot(spec_pharm[1, :])
plt.plot(
    baseline.t_mat[1, :, -3],
    baseline[1, :, -3],
    linewidth = 0.5, color = 'gray'
)
pltools.add_scalebar(y_units = 'mV', anchor = (0.5, 0.1), omit_x = True, y_size = 25)
plt.text(4500, -32, '$-30$mV', ha = 'right', va = 'top', size = 'small')
plt.xlim(Kslow_xlim[0], 4500)

if IMG_PATH is not None:
    plt.savefig(os.path.join(IMG_PATH, 'ser_pharmacology.png'))
    
plt.show()